In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "20"

import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import scanpy as sc
from hits.visualize import interactive
from bokeh.io import output_notebook
from hdbscan import HDBSCAN

from perturbseq import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

Loading BokehJS ...

In [ ]:
PREFIX = '20240310_RPE1_final_low_UMI_count_regressions_'

# Data loading and filtering droplets

In [ ]:
import scanpy as sc

In [ ]:
full_pop = sc.read_h5ad('/data/norman/southark/tfs_standardized/240308_RPE1_CRISPRa_final_population.h5ad')

In [ ]:
full_pop = CellPopulation(pd.DataFrame(full_pop.X.todense().A, index=full_pop.obs.index, columns=full_pop.var.index), full_pop.obs, full_pop.var, calculate_statistics=False)

In [ ]:
full_pop.cells['num_cells'].value_counts()

In [ ]:
full_pop.genes['in_matrix'] = True

# Fixing names

In [ ]:
feature_library = pd.read_csv('/lila/data/norman/southark/rpe1_tfs/rpe1_tfs_crispra_w_controls.csv')
feature_library['end'] = feature_library['sequence'].map(lambda x: x[-5:])
feature_library['name'] = feature_library['name'].replace({'non_targeting': 'non-targeting'})
feature_library['new_name'] = feature_library['name'] + '_' + feature_library['sequence']

In [ ]:
name_mapper = pd.Series(feature_library['new_name'].values, index=feature_library['id'])

In [ ]:
name_mapper

In [ ]:
control_guides = full_pop.cells.query('control')['guide_identity'].unique()
off_target_controls = np.setdiff1d(full_pop.cells.query('guide_target=="non"')['guide_identity'].unique(), control_guides)

In [ ]:
off_target_mapper = pd.Series(map(lambda x: x.replace('non_targeting', 'off-target'), off_target_controls), index=off_target_controls)

In [ ]:
feature_library['corrected_name'] = feature_library['id'].map(lambda x: off_target_mapper.get(x, x))
feature_library['corrected_name'] = feature_library['corrected_name'].map(lambda x: name_mapper.get(x, x))

feature_library = feature_library.set_index('corrected_name')

In [ ]:
name_mapper = pd.Series(feature_library.index, index=feature_library['id'])

In [ ]:
name_mapper

In [ ]:
control_reduced_name_mapper = name_mapper.copy()

In [ ]:
control_reduced_name_mapper.loc[control_reduced_name_mapper.str.contains('non-targeting')] = 'control'

In [ ]:
control_reduced_name_mapper

# UMI factors

In [ ]:
full_pop.cells['number_of_cells'] = full_pop.cells['num_cells']

In [ ]:
UMI_counts = full_pop.cells.groupby(['gem_group', 'number_of_cells'])['UMI_count'].median()
factors = (UMI_counts/UMI_counts.xs(1, level=1))

cell_factors = full_pop.cells.apply(lambda x: factors.loc[(x['gem_group'], x['number_of_cells'])], axis=1)

# Load identity calls and make design matrix

In [ ]:
guide_umis = pd.read_hdf('/data/norman/southark/tfs_standardized/240118_RPE1_CRISPRa_aggr_total_guide_umis.h5', key='guide_umis')

In [ ]:
guide_umis = pd.Series(guide_umis.values, 
    index=pd.MultiIndex.from_arrays([guide_umis.index.get_level_values(0), guide_umis.index.get_level_values(1).map(control_reduced_name_mapper)]))

In [ ]:
guide_umis = guide_umis.loc[full_pop.cells.index]
guide_umis = guide_umis[guide_umis>4].astype(bool).astype(np.float32)
guide_umis = guide_umis.groupby(level=[0,1]).sum()

In [ ]:
design_matrix = guide_umis.unstack(fill_value=0)

In [ ]:
cell_counts = design_matrix.sum(axis=0)

design_matrix = design_matrix.loc[:, cell_counts > 0]

In [ ]:
non_control_counts = design_matrix.loc[:, design_matrix.columns!='control'].sum(axis=1)

design_matrix['control'] = design_matrix['control'] + non_control_counts

In [ ]:
design_matrix = design_matrix.div(cell_factors, axis=0)

In [ ]:
design_matrix = design_matrix.loc[full_pop.cells.index]

# Regressions

In [ ]:
import warnings
from scipy.linalg import lstsq
from scipy.stats import t as t_stat
from joblib import Parallel, delayed

def compute_column(X, y, coef, ss_res, rank, XtX_inv, constant, gene_name):
    n = X.shape[0]
    
    # Degrees of freedom
    df_model = rank - constant
    df_resid = n - rank
    ss_tot = y @ y
        
    mse = ss_res / df_resid
    beta_var = mse * XtX_inv
    beta_se = np.sqrt(beta_var)

    # Compute T and p-values
    T = coef / beta_se
    pval = 2 * t_stat.sf(np.fabs(T), df_resid)

    out = pd.DataFrame([pd.Series(coef, index=X.columns, name='coef'),
                        pd.Series(pval, index=X.columns, name='p')]).T
    if gene_name is not None:
        out['gene_name'] = gene_name

    return out

def least_squares(X, Y, coef_only=False, constant=False, XtX_inv=None, n_jobs=-1):
    """
    Perform ordinary least squares regression.
    
    Parameters:
    - X: DataFrame containing the independent variables.
    - Y: DataFrame containing the dependent variables.
    - coef_only: If True, returns only the coefficients.
    - constant: Boolean, indicating if there's a constant term in the regression.
    - n_jobs: Number of cores to use in parallel. -1 means using all processors.
    
    Returns:
    - Dictionary where each key corresponds to a dependent variable's column name and its associated value is the DataFrame containing coefficients and p-values for that dependent variable.
    """
    n = X.shape[0]

    # FIT LEAST SQUARES REGRESSION
    coef, ss_res, rank, _ = lstsq(X, Y, cond=None, check_finite=False)
    
    calc_ss_res = False
    if rank < X.shape[1]:
        warnings.warn(
            "Design matrix supplied with `X` parameter is rank "
            f"deficient (rank {rank} with {X.shape[1]} columns). "
            "That means that one or more of the columns in `X` "
            "are a linear combination of one or more of the "
            "other columns."
        )
        calc_ss_res = True
        
    if coef_only:
        return coef

    if XtX_inv is None:
        XtX_inv = (np.linalg.pinv(X.T @ X).diagonal())

    if calc_ss_res:
        resid = Y.values - X.values @ coef
        ss_res = (resid**2).sum(axis=0)
    
    ss_res = pd.Series(ss_res, index=Y.columns)

    results = Parallel(n_jobs=n_jobs, verbose=10)(
        delayed(compute_column)(X, Y[col], coef[:, i], ss_res[col], rank, XtX_inv, constant, col)
        for i, col in enumerate(Y.columns)
    )
    
    results = pd.concat(results).pivot(columns='gene_name')
    return results['coef'], results['p']

In [ ]:
coefs, ps = least_squares(design_matrix, full_pop.matrix,
                          coef_only=False, n_jobs=12)

In [ ]:
from perturbseq.differential_expression import _multi_test_correct

coefs.to_hdf(PREFIX + 'coefs.hdf', key='coefs')
ps.to_hdf(PREFIX + 'ps.hdf', key='ps')
adj_ps = ps.dropna(axis=1).T.apply(lambda x: _multi_test_correct(x, 0.05, 'fdr_bh')).T
adj_ps.to_hdf(PREFIX + 'adj_ps.hdf', key='adj_ps')

# Saving results

In [ ]:
count_profiles = dict()
naive_mean_profiles = dict()
frac_profiles = dict()

for perturbation, col in tqdm(design_matrix.iteritems(), total=design_matrix.shape[1]):
    mask_col = col > 0
    naive_mean_profiles[perturbation] = full_pop.matrix.loc[mask_col].mean()
    count_profiles[perturbation] = (full_pop.matrix.loc[mask_col] > 0).sum()
    frac_profiles[perturbation] = (full_pop.matrix.loc[mask_col] > 0).mean()
    
count_profiles = pd.DataFrame(count_profiles).T
naive_mean_profiles = pd.DataFrame(naive_mean_profiles).T
frac_profiles = pd.DataFrame(frac_profiles).T

In [ ]:
count_profiles.to_hdf(PREFIX + 'count_profiles.hdf', key='count_profiles')
naive_mean_profiles.to_hdf(PREFIX + 'naive_mean_profiles.hdf', key='naive_mean_profiles')
frac_profiles.to_hdf(PREFIX + 'frac_profiles.hdf', key='frac_profiles')

In [ ]:
mean_pop = MeanPopulation(coefs,
                          pd.DataFrame(cell_counts, columns=['cell_count']),
                          full_pop.genes.loc[coefs.columns], calculate_statistics=False)

mean_pop.genes.index.name = 'gene_id'
mean_pop.normalized_matrix = dict()
mean_pop.normalized_matrix['count_profile'] = count_profiles
mean_pop.normalized_matrix['naive_mean_profile'] = naive_mean_profiles

mean_pop.normalized_matrix['p'] = ps
mean_pop.normalized_matrix['adj_p'] = adj_ps

mean_pop.to_hdf(PREFIX + 'coef_mean_pop.hdf', store_normalized_matrix=True)